In [0]:
#%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.utils.data as data_utils
from utils import epoch, epoch_robust_bound, Flatten

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

In [5]:
mnist_train = datasets.MNIST("./", train=True, download=True, transform=transforms.ToTensor())

Processing...
Done!


In [0]:
mnist_test = datasets.MNIST("./", train=False, download=True, transform=transforms.ToTensor())

In [0]:
train_loader = DataLoader(mnist_train, batch_size = 100, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size = 100, shuffle=False)

# Model

In [0]:
model_cnn_medium = nn.Sequential(nn.Conv2d(1, 32, 3, padding=0, stride=1), nn.ReLU(),
                                   nn.Conv2d(32, 32, 4, padding=0, stride=2), nn.ReLU(),
                                   nn.Conv2d(32, 64, 3, padding=0, stride=1), nn.ReLU(),
                                   nn.Conv2d(64, 64, 4, padding=0, stride=2), nn.ReLU(),
                                   Flatten(),
                                   nn.Linear(64*4*4, 512), nn.ReLU(),
                                   nn.Linear(512, 512), nn.ReLU(),
                                   nn.Linear(512, 10)).to(device)

# Training

In [0]:
opt = optim.Adam(model_cnn_medium.parameters(), lr=1e-3)
EPSILON = 0.1
EPSILON_TRAIN = 0.2
epsilon_schedule = [0.0, 0.01, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2] + 80*[EPSILON_TRAIN]
kappa_schedule = 2*[1]
for i in range(98): kappa_schedule.append(1-0.005*i) 

print("Epsilon   ", "Robust Loss", "Test Err", "Test Robust Err", sep="\t")
for t in range(len(epsilon_schedule)):
    _, robust_loss = epoch_robust_bound(train_loader, model_cnn_medium, epsilon_schedule[t], device, kappa_schedule[t], opt)
    
    # check loss and accuracy on test set
    test_err, test_loss = epoch(test_loader, model_cnn_medium, device)
    adv_err, adv_loss = epoch_robust_bound(test_loader, model_cnn_medium, EPSILON, device, kappa_schedule[t])
    
    
    if t == 25:  #decrease learning rate after 25 epochs
      for param_group in opt.param_groups:
        param_group["lr"] = 1e-4
    
    if t == 41:  #decrease learning rate after 41 epochs
      for param_group in opt.param_groups:
        param_group["lr"] = 1e-5
    
    print(*("{:.6f}".format(i) for i in (epsilon_schedule[t], robust_loss, test_err, adv_err)), sep="\t")

Epsilon   	Robust Loss	Test Err	Test Robust Err
0.000000	0.200056	0.017300	1.000000
0.010000	3425.416996	0.014600	1.000000
0.030000	16847.909989	0.013600	1.000000
0.040000	97.369732	0.047600	1.000000
0.050000	7.203265	0.033300	1.000000
0.060000	3.029905	0.029500	0.997600
0.070000	1.907495	0.028400	0.975300
0.080000	1.501102	0.026600	0.710500
0.090000	1.264801	0.025600	0.320300
0.100000	1.121258	0.025100	0.252200
0.110000	1.027579	0.024200	0.190700
0.120000	0.961214	0.024100	0.158100
0.130000	0.913403	0.023300	0.139500
0.140000	0.880165	0.024600	0.122400
0.150000	0.856929	0.024500	0.102600
0.160000	0.818503	0.025200	0.099900
0.170000	0.803045	0.025500	0.093400
0.180000	0.786592	0.025800	0.083700
0.190000	0.784770	0.026200	0.082800
0.200000	0.779713	0.026700	0.071700
0.200000	0.722564	0.025500	0.075200
0.200000	0.684360	0.024600	0.066400
0.200000	0.651361	0.025100	0.066500
0.200000	0.625512	0.024500	0.065300
0.200000	0.603710	0.024400	0.061600
0.200000	0.586140	0.024900	0.062600
0.200000